In [1]:
import pandas as pd
import random
import yaml
import json

data = pd.read_csv("safegraph_cbg_population_estimate.csv")
display(data.head())

,census_block_group,B00001e1,B00001m1,B00002e1,B00002m1
0,10010201001,72.0,-1.0,32.0,-1.0
1,10010201002,173.0,-1.0,66.0,-1.0
2,10010202001,100.0,-1.0,43.0,-1.0
3,10010202002,167.0,-1.0,54.0,-1.0
4,10010203001,184.0,-1.0,80.0,-1.0


In [8]:
STATE_FIPS = "26"

# Filter the data for the specified state
cbgs = list(data['census_block_group'])
states = [str(cbg)[:2] for cbg in cbgs]
state_data = [cbg for cbg, state in zip(cbgs, states) if state == STATE_FIPS]
print(len(state_data))

# print list of counties in the state
counties = set([str(cbg)[2:5] for cbg in state_data])
print(len(counties), "counties in the data")
print(counties)


8205
83 counties in the data
{'165', '095', '045', '105', '049', '075', '091', '135', '077', '163', '007', '023', '029', '009', '061', '139', '003', '129', '059', '113', '119', '111', '057', '157', '039', '159', '115', '041', '055', '101', '109', '125', '051', '019', '127', '021', '103', '083', '133', '005', '073', '053', '025', '027', '141', '011', '063', '067', '131', '013', '117', '149', '069', '093', '087', '065', '047', '161', '001', '035', '099', '143', '123', '089', '079', '043', '153', '031', '121', '147', '097', '085', '071', '137', '151', '015', '033', '107', '081', '145', '037', '155', '017'}


In [12]:
COUNTY_FIPS = ["099"]

# Pick 80% of the cbgs from each county
county_data = [cbg for cbg in state_data if str(cbg)[2:5] in COUNTY_FIPS]
# shuffle the data
random.shuffle(county_data)
# pick random portion of the data  
county_data = county_data[:int(1*len(county_data))]
# filter out any rows with empty values
county_data = [cbg for cbg in county_data if not pd.isnull(data[data['census_block_group'] == cbg]['B00001e1'].values[0])]
# print the number of cbgs
print(len(county_data))
# print the total population
print(int(sum(data[data['census_block_group'].isin(county_data)]['B00001e1'])))


624
70799


In [13]:
# Create dictionary of cbgs in county_data and their population
cbg_pop = {cbg: data[data['census_block_group'] == cbg]['B00001e1'].values[0] for cbg in county_data}
# convert populations to integers
cbg_pop = {str(cbg): int(pop) for cbg, pop in cbg_pop.items()}
cbg_pop = {"GEOIDs": cbg_pop}

# Write the dictionary to a yaml file
with open("CZ_output_test.yaml", "w") as f:
    yaml.dump(cbg_pop, f)